# 請結合前面的知識與程式碼，比較不同的 optimizer 與 learning rate 組合對訓練的結果與影響
### 常見的 optimizer 包含
* SGD
* RMSprop
* AdaGrad
* Adam

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [8]:
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [20]:
## 超參數設定
"""
Set your required experiment parameters
"""
LEARNING_RATE = 1e-1
EPOCHS = 20
BATCH_SIZE = 256
MOMENTUM = 0.95
#opt = ['SGD','RMSprop','AdaGrad','Adam']
#opt = ['keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)','keras.optimizers.RMSprop(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)','keras.optimizers.AdaGrad(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)','keras.optimizers.Adam(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)']

In [26]:
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
for i in range(4):
    keras.backend.clear_session() # 把舊的 Graph 清掉
    #print("Experiment with LR = %.6f" % (x))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    if i==0:
        optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    elif i==1:
        optimizer = keras.optimizers.RMSprop(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    elif i==2:
        optimizer = keras.optimizers.AdaGrad(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    else :
        optimizer = keras.optimizers.Adam(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["accuracy"]
    valid_acc = model.history.history["val_accuracy"]
    
    exp_name_tag = "exp-lr-%s" % str(x)
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}


SyntaxError: invalid syntax (<ipython-input-26-33d24f69d789>, line 12)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

"""
將實驗結果繪出
"""